In [1]:
import pprint, json, math, os, sys
import fitz
import pandas as pd
from collections import defaultdict

dir_path = "C:\\Users\\Kaustubh.keny\\OneDrive - Cogencis Information Services Ltd\\Documents\\mywork-repo\\"
fund_path = "C:\\Users\\Kaustubh.keny\\OneDrive - Cogencis Information Services Ltd\\Documents\\Dec 24\\"

# dir_path = "C:\\Users\\rando\\OneDrive\\Documents\\mywork-repo"
# fund_path =  "C:\\Users\\rando\\OneDrive\\Documents\\Dec 24"
sys.path.append(os.path.abspath(dir_path))

from app.utils import Helper
from app.fund_regex import *

dry_path = r'\data\output\DryRun.pdf'
fin_path = r'\data\input\financial_indices.xlsx'
mutual_fund = Helper.get_fund_paths(fund_path)

In [3]:
def extract_clipped_data(input:str, pages:list, bboxes:list):
        
        document = fitz.open(input)
        final_list = []
    
        for pgn in pages:
            page = document[pgn]
            
            all_blocks = [] #store every data from bboxes
            
            for bbox in bboxes:
                blocks, seen_blocks = [], set()  #store unique blocks based on content and bbox
                
                page_blocks = page.get_text('dict', clip=bbox)['blocks']
                for block in page_blocks:
                    if block['type'] == 0 and 'lines' in block: #type 0 means text block
                        #hash_key
                        block_key = (tuple(block['bbox']), tuple(tuple(line['spans'][0]['text'] for line in block['lines'])))
                        if block_key not in seen_blocks:
                            seen_blocks.add(block_key)
                            blocks.append(block)

                sorted_blocks = sorted(blocks, key=lambda x: (x['bbox'][1], x['bbox'][0]))
                all_blocks.append(sorted_blocks)

            final_list.append({
                "pgn": pgn,
                "block": all_blocks #will be list[list,list,..]
            })

        document.close()
        return final_list

In [4]:
def extract_data_relative_line(path: str, line_x: float, side: str):
    doc = fitz.open(path)
    pages = doc.page_count

    final_list = []

    for pgn in range(pages):
        page = doc[pgn]

        blocks = page.get_text("dict")["blocks"]
        sorted_blocks = sorted(blocks, key=lambda x: (x["bbox"][1], x["bbox"][0]))
        extracted_blocks = []

        # Keep track of blocks to avoid duplicates
        added_blocks = set()

        for block in sorted_blocks:
            block_id = id(block)  # Unique identifier for the block

            for line in block.get("lines", []):
                for span in line.get("spans", []):
                    origin = span["origin"]
                    x0, _ = origin

                    # Check the side condition
                    if side == "left" and x0 < line_x and block_id not in added_blocks:
                        extracted_blocks.append(block)
                        added_blocks.add(block_id)  # Mark block as added
                    elif side == "right" and x0 > line_x and block_id not in added_blocks:
                        extracted_blocks.append(block)
                        added_blocks.add(block_id)  # Mark block as added

      
        final_list.append({
            "pgn": pgn,
            "blocks": extracted_blocks
        })

    doc.close()

    return final_list

def extract_spans(data):
    final_span = []
    for pgn,blocks in enumerate(data):
        spans = []
        print(f"___{pgn}___")
        for num,block in enumerate(blocks.get('blocks',[])):
            for line in block.get("lines", []):
                for span in line.get("spans", []):
                    if num in range(3,10):
                        print(span['text'], span['size'])
        final_span.append(spans)
    return final_span

def get_proper_fund_names(path:str,pages:list):
        
    doc = fitz.open(path)
    final_fund_names = dict()
    
    for pgn in range(doc.page_count):
        text_all = ''
        if pgn in pages:
            # print(pgn)
            page = doc[pgn]            
            blocks = page.get_text("dict")['blocks']
            
            sorted_blocks = sorted(blocks,key=lambda k:(k['bbox'][1],k['bbox'][0]))
            for count,block in enumerate(sorted_blocks):
                for line in block.get("lines", []):
                    for span in line.get("spans", []):
                        text = span['text'].strip()
                        if count in range(0,1):
                            text_all+=f" {text}"
            print(text_all)
        pattern = r"MIRAE ASSET.*?\b(?:ETF|EOF|FOF|FTF|FUND)\b"
        if matches := re.findall(pattern, text_all.strip(), re.DOTALL):
            final_fund_names[pgn] = matches[0]
        else:
            final_fund_names[pgn] = ""
    return final_fund_names
  
def get_clipped_data(input:str, bboxes:list[set]):
    
        document = fitz.open(input)
        final_list = []
        
        for pgn in range(document.page_count):
            page = document[pgn]

            blocks = []
            for bbox in bboxes:
                blocks.extend(page.get_text('dict', clip = bbox)['blocks']) #get all blocks
            
            filtered_blocks = [block for block in blocks if block['type']== 0 and 'lines' in block]
            # sorted_blocks = sorted(filtered_blocks, key= lambda x: (x['bbox'][1], x['bbox'][0]))
            
            final_list.append({
            "pgn": pgn,
            "block": filtered_blocks
            })
            
            
        document.close()
        return final_list
    
def get_clipped_text(input:str, bboxes:list[set]):

    document = fitz.open(input)
    final_list = []
    
    for pgn in range(document.page_count):
        page = document[pgn]
        blocks = []
        for bbox in bboxes:
            blocks = page.get_text('text', clip = bbox).split('\n') #get all blocks
  
        final_list.append({
        "pgn": pgn,
        "block": blocks
        })   
    document.close()
    return final_list

In [3]:
sample_path  = mutual_fund['DSP Mutual Fund']
pages = [12, 14,16]
bboxes = [(200, 50, 380, 812),(380, 50, 580, 812)]

In [29]:
df = extract_clipped_data(sample_path,pages,bboxes)

In [ ]:
lines = [
    ((220, 0), (220, 812)),# Vertical line
    #((420, 0), (420, 1000))
]

rectangles = [(0, 50, 200, 812),(200, 50, 380, 812),(380, 50, 580, 812)]
pages = [i for i in range(1,110)]
Helper.draw_lines_on_pdf(sample_path, lines, rectangles, pages, dir_path +dry_path)


Modified PDF saved to: C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\mywork-repo\data\output\DryRun.pdf


In [4]:
data = Helper.get_all_pdf_data(sample_path)

In [6]:
data[100:118]

[{'pgn': 100,
  'blocks': [{'number': 76,
    'type': 0,
    'bbox': (979.602783203125,
     39.50933074951172,
     1156.2506103515625,
     56.29533386230469),
    'lines': [{'spans': [{'size': 14.0,
        'flags': 16,
        'font': 'ZurichBoldBT',
        'color': -1892569,
        'ascender': 0.9629999995231628,
        'descender': -0.23600000143051147,
        'text': 'Snapshot of Fund of Funds',
        'origin': (979.602783203125, 52.9913330078125),
        'bbox': (979.602783203125,
         39.50933074951172,
         1156.2506103515625,
         56.29533386230469)}],
      'wmode': 0,
      'dir': (1.0, 0.0),
      'bbox': (979.602783203125,
       39.50933074951172,
       1156.2506103515625,
       56.29533386230469)}]},
   {'number': 1,
    'type': 0,
    'bbox': (52.63690185546875,
     64.69470977783203,
     339.7698059082031,
     100.68470764160156),
    'lines': [{'spans': [{'size': 10.0,
        'flags': 16,
        'font': 'ZurichBoldBT',
        'color': -1,


In [ ]:
def get_something(path: str):
    pattern = r"^(REDEMPTION PROCEEDS|FEATURES|ASSET ALLOCATION|FUND MANAGER|SCHEME|Sr\. No\.)$"

    with fitz.open(path) as doc:
        for dpgn, page in enumerate(doc):
                page_blocks = page.get_text("dict")["blocks"]
                sorted_blocks = sorted(page_blocks, key=lambda x: (x["bbox"][1], x["bbox"][0]))

                for block_count, block in enumerate(sorted_blocks):
                    if "lines" not in block:
                        continue
                    for line in block["lines"]:
                        for span in line["spans"]:
                            text = span["text"].strip()
                            if re.match(pattern, text):
                                print(text)

In [80]:
import camelot

In [94]:
def get_something(path: str):
    pattern = r"^(REDEMPTION PROCEEDS|FEATURES|ASSET ALLOCATION|FUND MANAGER|SCHEME|Sr\. No\.)$"

    with fitz.open(path) as doc:
        exists = defaultdict(int)
        for pgn, page in enumerate(doc):
            page_text = [t.strip() for t in page.get_text().split("\n")]
            for text in page_text:
                if re.match(pattern,text):
                    exists[pgn]+=1
                    
    
    return [str(pgn+1) for pgn, count in exists.items() if count > 4]                  
    

In [95]:
pages = get_something(sample_path)

In [ ]:
pages

In [ ]:
imp_pages = ",".join(pages)
tables = camelot.read_pdf(sample_path,pages=imp_pages, flavor="stream", table_areas = ["30,50,612,812"])

In [204]:
excel_path = r'C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\mywork-repo\notebook\example.xlsx'

In [213]:
with pd.ExcelWriter(excel_path, engine="openpyxl") as writer:
    for i, table in enumerate(tables):
        df = table.df  # Convert table to DataFrame
        df = df.iloc[1:,:]
        df.columns = df.iloc[0,:]
        df = df.iloc[1:,:]
        sheet_name = f"Table_{i+1}"  # Naming each sheet
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"Excel file saved at: {excel_path}")

Excel file saved at: C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\mywork-repo\notebook\example.xlsx
